In [ ]:
%matplotlib widget

from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd
from scipy.signal import medfilt

from fimpylab import LightsheetExperiment

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, fictive_trajectory_and_fit, fit_phase_neurons, \
    qap_sorting_and_phase
from circle_fit import hyper_fit

In [ ]:
plt.close("all")
path = Path("/Users/luigipetrucco/Desktop/batch_210908/no/210908_f1b_cwccw")
traces = fl.load(path / "filtered_traces.h5", "/detr")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
anat = fl.load(path / "data_from_suite2p_unfiltered.h5", "/anatomy_stack")
traces[np.isnan(traces)] = 0

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LightsheetExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

t_start = 150
time_slices_dict = dict(clol=(t_start*fn, 8000), cwccw=(t_start*fn, traces.shape[0] // 2))
t_lims = time_slices_dict[path.name.split("_")[-1]]
t_slice = slice(*t_lims)

In [ ]:
suite2p_idxs = np.array(fl.load(path / "data_from_suite2p_unfiltered.h5", "/suite2p_idxs"))

In [ ]:
np.argwhere(suite2p_idxs == 428)

In [ ]:
diff = np.concatenate([medfilt(np.diff(traces[:, suite2p_idxs == 428].flatten()), 51)*100, [0]])

In [ ]:
#cc = np.corrcoef(np.concatenate([diff[:, np.newaxis], traces], 1).T)
np.fill_diagonal(cc, np.nan)

In [ ]:
plt.figure(figsize=(4, 2))
cc_wnd = 400
i_array = np.arange(t_slice.start, t_slice.stop, cc_wnd*fn)
cc_mats = np.zeros((traces.shape[1], traces.shape[1], len(i_array)))

for n, i in enumerate(i_array):
    cc_mats[:, :, n] = np.corrcoef(traces[i:i + cc_wnd*fn, :].T)
corr_mat = np.nanmean(cc_mats, 2)
prev_mat = corr_mat.copy()
plt.hist(corr_mat.flatten(), np.arange(-1, 1, 0.05), alpha=0.5, lw=0)

n_pairs = 50
selected = []
incl_cc = []
for n in range(n_pairs):
    pair = np.unravel_index(np.nanargmin(corr_mat), corr_mat.shape)
    selected.append(pair)
    incl_cc.append(corr_mat[pair[0], pair[1]])
    for p in pair:
        corr_mat[p, :] = np.nan
        corr_mat[:, p] = np.nan
# np.array((selected))

plt.hist(corr_mat.flatten(), np.arange(-1, 1, 0.05), alpha=0.5, lw=0)
plt.yscale("log")

In [ ]:
pairs = []

In [ ]:
corr_mat

In [ ]:
2, 12, 20, 22, 23

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(traces[:, 428])
i = 0
plt.plot(traces[:, selected[i][0]])
plt.plot(traces[:, selected[i][1]])
#plt.plot(diff)
# plt.plot(traces[:, np.nanargmin(corr_mat[:, 428])])
plt.xlim(0, 2000)

In [ ]:
selected_comb = []
[selected_comb.extend(selected[i]) for i in range(50)]# [0, 2, 12, 20, 22, 23]]
a=0

In [ ]:
selected_comb

In [ ]:
# plt.close("all")
pcaed, phase = pca_and_phase(traces[:4000, selected_comb], traces[:4000, selected_comb])
mot_t_slice = slice(traces.shape[0] // 2, traces.shape[0])
f, axs = plt.subplots(1, 1, figsize=(5., 5.), sharex=True, sharey=True)
# plt.subplot(1,2,1)
phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
axs.plot(pcaed[:, 0], pcaed[:, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs.scatter(pcaed[:, 0], pcaed[:, 1], 
                 c=phase, lw=0.5, s=5, cmap="twilight",) 
axs.axes.set_aspect("equal")